In [93]:
from fpylll import IntegerMatrix, LLL
import numpy as np

In [94]:
from sage.all import Matrix, vector, identity_matrix, zero_matrix, ZZ

In [95]:
q = 101  # Replace with your desired modulus (q must be prime for a field)
# Zq = GF(q)
Zq = GF(q)

In [96]:
A = Matrix(Zq, [
    [64, 6, 77],
    [57, 89, 82],
    [24, 3, 91],
    [14, 99, 63],
    [18, 30, 89],
    [35, 22, 86],
    [77, 55, 76]
])
m, n = A.dimensions()
b = vector(Zq, [76, 27, 19, 77, 91, 61, 46])

In [97]:
A = Matrix(Zq, [
    [64, 6, 77],
    [57, 89, 82],
    [21, 3, 91],
    [14, 91, 63],
    [18, 30, 29],
    [35, 22, 86],
    [77, 55, 76]
])
m, n = A.dimensions()

In [98]:
s = vector(Zq, [1, 2, 3])
e = vector(Zq, [1, 0, -1, 0, 1, 0, -1])
b = A * s + e

In [111]:
def primal_attack(A, b):
    # Defining the matrix w for LLL reduction
    A_transpose = A.transpose()
    A_prime = A_transpose.rref().transpose()
    
    zero_row = Matrix(Zq, 1, n, [0, 0, 0])
    
    w = A_prime.stack(zero_row)
    w = w.augment(vector(Zq, list(b)+[1]))
    
    ll = q * identity_matrix(m - n)
    lu = zero_matrix(n, m - n)
    l = lu.stack(ll).stack(zero_matrix(1, m-n))
    
    w = w.change_ring(ZZ)
    w = w.augment(l)

    # permute the rows of w randomly
    permutation = np.random.permutation(w.nrows()) + 1
    permutation = Permutation(permutation)
    
    b = [x for x in b] + [1]
    print(permutation)
    print(b)
    b_perm = vector([b[int(i-1)] for i in permutation])
    w.permute_rows(permutation)
    print(w)

    w_transpose = w.transpose()
    w_transpose.LLL()
    
    # Perform LLL reduction and solve for s and e
    e = w_transpose.LLL()[0][:-1]
    s_pred = A.solve_right(b_perm - e)
    return s_pred, e

In [112]:
primal_attack(A, b)

[7, 8, 4, 1, 3, 5, 2, 6]
[5, 77, 97, 82, 65, 34, 10, 1]
[ 79  65  53  10   0   0   0 101]
[  0   0   0   1   0   0   0   0]
[ 28  17  76  82 101   0   0   0]
[  1   0   0   5   0   0   0   0]
[  0   0   1  97   0   0   0   0]
[ 97  58  48  65   0 101   0   0]
[  0   1   0  77   0   0   0   0]
[ 82  10  18  34   0   0 101   0]


TypeError: unsupported operand parent(s) for -: 'Vector space of dimension 8 over Finite Field of size 101' and 'Ambient free module of rank 7 over the principal ideal domain Integer Ring'

In [9]:
A_transpose = A.transpose()
rref_a_transpose = A_transpose.rref()[0]

In [10]:
A_prime = A_transpose.rref().transpose()
A_prime 

[ 1  0  0]
[ 0  1  0]
[ 0  0  1]
[28 17 76]
[97 58 48]
[82 10 18]
[79 65 53]

In [11]:
zero_row = Matrix(Zq, 1, n, [0, 0, 0])

In [12]:
w = A_prime.stack(zero_row)

In [13]:
w = w.augment(vector(Zq, list(b)+[1]))

In [14]:
w

[ 1  0  0  5]
[ 0  1  0 77]
[ 0  0  1 97]
[28 17 76 82]
[97 58 48 65]
[82 10 18 34]
[79 65 53 10]
[ 0  0  0  1]

In [15]:
ll = q * identity_matrix(m - n)
lu = zero_matrix(n, m - n)

In [16]:
l = lu.stack(ll).stack(zero_matrix(1, m-n))
l

[  0   0   0   0]
[  0   0   0   0]
[  0   0   0   0]
[101   0   0   0]
[  0 101   0   0]
[  0   0 101   0]
[  0   0   0 101]
[  0   0   0   0]

In [17]:
w, l

(
[ 1  0  0  5]  [  0   0   0   0]
[ 0  1  0 77]  [  0   0   0   0]
[ 0  0  1 97]  [  0   0   0   0]
[28 17 76 82]  [101   0   0   0]
[97 58 48 65]  [  0 101   0   0]
[82 10 18 34]  [  0   0 101   0]
[79 65 53 10]  [  0   0   0 101]
[ 0  0  0  1], [  0   0   0   0]
)

In [18]:
w = w.change_ring(ZZ)

In [19]:
w

[ 1  0  0  5]
[ 0  1  0 77]
[ 0  0  1 97]
[28 17 76 82]
[97 58 48 65]
[82 10 18 34]
[79 65 53 10]
[ 0  0  0  1]

In [20]:
w = w.augment(l)

In [24]:
w_transpose

[  1   0   0  28  97  82  79   0]
[  0   1   0  17  58  10  65   0]
[  0   0   1  76  48  18  53   0]
[  5  77  97  82  65  34  10   1]
[  0   0   0 101   0   0   0   0]
[  0   0   0   0 101   0   0   0]
[  0   0   0   0   0 101   0   0]
[  0   0   0   0   0   0 101   0]

In [25]:
w_transpose = w.transpose()
w_transpose.LLL()

[  1   0  -1   0   1   0  -1   1]
[  2  -2   9  -1   5   3   0   0]
[  6   2   0   0  -9   7  -2   0]
[  4   4   5   2   4 -10   6   1]
[  0   9   6   3   2  -4  -6   0]
[ -3  -6   1   6   0  -5  -8  -6]
[ -1  -5  -4   7   7   5   4  -8]
[ -5  -5   0  10   0   1   0   7]

In [23]:
e = w_transpose.LLL()[0][:-1]
A.solve_right(b - e)

(1, 2, 3)